# First slide

This is my first slide.

$$ f(x) = x^2 $$

In [1]:
from kingdon import Algebra

alg = Algebra(2, 0, 1)
locals().update(alg.blades)

In [2]:
# Construct points and lines from coefficients
point = lambda x, y: (e0 + x*e1 + y*e2).dual()
line = lambda a, b, c: a*e1 + b*e2 + c*e0

# The formulas for projection and rejection are the same for all elements of geometry in PGA
project = lambda a, b: (a | b) / b
reject = lambda a, b: (a | b)

In [3]:
A = point(1, 1)
B = point(-1, 1)
C = point(-1, -1)
AC = A & C

In [4]:
alg.graph(
    0xD0FFE1, [A,B,C],
    0x882288, project(B, AC), "project B onto AC",
    0x882288, project(AC, B), "project AC onto B",
    0x008844, reject(AC, B),  "reject AC from B",
    0x00AA88, AC, "AC",
    0x224488, A, "A", B, "B", C, "C",
    lineWidth=3, grid=1, labels=1
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …